# Structure of the State Dictionary

In this tutorial, we will go over the different types of states used in MotorNet, how they flow during simulation, how they are structured and what information they carry.

Let's start by importing what we need.


In [1]:

import os
import sys
import numpy as np
import torch as th
from IPython import get_ipython

motornet_in_cwd = os.path.exists("MotorNet") or os.path.exists("motornet")
colab_env = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
colab_initialized = True if motornet_in_cwd else False

if colab_env and not colab_initialized:
  !pip install gymnasium>=2.8
  !pip install git+https://github.com/OlivierCodol/MotorNet@pytorch
  sys.path.append('MotorNet')
  print("Running cell using COLAB initialization...")
elif colab_env and colab_initialized:
  print("Already initialized using COLAB initialization.")
else:
  paths = [p for p in sys.path if os.path.exists(p)]
  local_initialized = True if [p for p in paths if "MotorNet" in os.listdir(p)] else False
  if local_initialized:
    %load_ext autoreload
    %autoreload 2
    print("Already initialized using LOCAL initialization.")
  else:
    path = [p for p in paths if p.__contains__("examples")]
    if len(path) != 1:
      raise ValueError("Path to MotorNet could not be determined with certainty.")
    else:
      path = path[0]
    sys.path.append(os.path.dirname(path[:path.rfind('examples')]))
    %load_ext autoreload
    %autoreload 2
    print("Running cell using LOCAL initialization...")


import motornet as mn


print('All packages imported.')
print('pytorch version: ' + th.__version__)
print('numpy version: ' + np.__version__)
print('motornet version: ' + mn.__version__)

Running cell using LOCAL initialization...
All packages imported.
pytorch version: 2.0.1
numpy version: 1.23.0
motornet version: 0.2.0



# I. Types of States

States are the main mean of communication between MotorNet objects and so they convey a wide variety of information.

There are 5 states that will always be present at the `Effector` level, regardless of the effector being created.

- Joint state
- Cartesian state
- Muscle state
- Geometry state
- Fingertip

Additionally, `Environment` objects will likely have states associated with its computation. This will be further detailed below in this tutorial, as well as in the follow-up `3-environments.ipynb` tutorial.



# II. State flow at runtime

Below is an overview of where states are generated and updated in a MotorNet model, and how they flow from object to object.




<img src="img/states.png" alt="drawing" width="600"/>


From the above illustration, we can clearly see that `Muscle` objects generate muscle states, `Skeleton` objects generate joint, cartesian, and fingertip states, and `Effector` objects generate geometry states (though partially using `Skeleton` information). The `Environment` object applies noise and time delays to states if applicable, creates an observation vector using these states, and outputs all the state contents alongside `Environment` specific information so that the user can collect them and feed them into the policy network if desired.


# III. Dimensionality of State tensors

## III. 1. Effector States
Let us create an effector, and then get initial states for a batch size of 7.
We use a pre-built effector with 4 muscle, and then add a fifth muscle. The effector's skeleton has 2 degrees of freedom and evolves in a 2D cartesian space.

One can retrieve the current states of an effector by using the `Effector.states` attribute, which returns a dictionary with all the states as entries.


In [2]:

effector = mn.effector.ReluPointMass24()

# adding a fifth muscle
effector.add_muscle(path_fixation_body=[0, 1], path_coordinates=[[1, 0], [0, 0]], max_isometric_force=1)

effector.reset(options={"batch_size": 7})

for key, state in effector.states.items():
  print(key + " shape: " + " " * (10-len(key)), state.shape)


joint shape:       torch.Size([7, 4])
cartesian shape:   torch.Size([7, 4])
muscle shape:      torch.Size([7, 4, 5])
geometry shape:    torch.Size([7, 4, 5])
fingertip shape:   torch.Size([7, 2])


From the example above it is fairly easy to see that for all states, the first dimension always corresponds to the `batch_size`. We can also see that for the muscle and geometry states, the last dimension is always the number of muscles.

The second dimension is the number of features of that state. For the joint state, this is the number of degrees of freedom times two (position and velocity). For the cartesian state, this is the dimensionality of the cartesian space (here 2D) times two (cartesian position of the effector's endpoint, cartesian velocity of the effector's endpoint). For the fingertip state, it is simply the cartesian position of the effector's endpoint, i.e., the first half of the cartesian state. This is for convenience of use, as we often want the cartesian position but not the velocities, for instance when penalizing positional error.

For the geometry state, this will always be musculotendon length, musculotendon velocity, and the moment of the muscle considered for each joint. Because there are two degrees of freedom here for the skeleton (two joints), there are two moments. Additionally, this information can be obtained by checking the `Plant` object's `geometry_state_name` attribute.

For the muscle state, this depends on the muscle type being used. This information can be obtained by checking the `Muscle` object's `state_name` attribute. The `Muscle` object is directly accessible from the `Plant` object as demonstrated below.



In [3]:
features = effector.muscle.state_name
for n, feature in enumerate(features):
  print("feature " + str(n) + ": ", feature)


feature 0:  activation
feature 1:  muscle length
feature 2:  muscle velocity
feature 3:  force



And below, we fetch the geometry state names using the equivalent attribute for geometry at the `Plant` level.


In [4]:
features = effector.geometry_state_name
for n, feature in enumerate(features):
  print("feature " + str(n) + ": ", feature)


feature 0:  musculotendon length
feature 1:  musculotendon velocity
feature 2:  moment for joint 0
feature 3:  moment for joint 1


## III. 2. Environment "States" (`obs` vector and `info` dictionary)

Let's now look at states at the `Environment` level. First, we build a `Environment` that includes the same effector as earlier (with five muscles).

We must first initialize the environment by using the `.reset()` method. Then we can print the observation and state shapes that this method returns.



In [5]:

effector = mn.effector.ReluPointMass24()

# adding a fifth muscle
effector.add_muscle(path_fixation_body=[0, 1], path_coordinates=[[1, 0], [0, 0]], max_isometric_force=1)

env = mn.environment.Environment(effector=effector, proprioception_delay=0.03, vision_delay=0.09)



obs, info = env.reset(options={"batch_size": 7})

print("obs shape:          ", obs.shape, end="\n\n\n")

for key, val in info.items():
  if type(val) is dict:
    print(key + ": ")
    for k, v in val.items():
      print("\t\t\t" + k + " shape:" + " " * (10-len(k)), v.shape)
  else:
    print(key + " shape:" + " " * (13-len(key)), val.shape)


obs shape:           torch.Size([7, 14])


states: 
			joint shape:      torch.Size([7, 4])
			cartesian shape:  torch.Size([7, 4])
			muscle shape:     torch.Size([7, 4, 5])
			geometry shape:   torch.Size([7, 4, 5])
			fingertip shape:  torch.Size([7, 2])
action shape:        torch.Size([7, 5])
noisy action shape:  torch.Size([7, 5])
goal shape:          torch.Size([7, 2])



We can see that the `Effector` states are carried over to the `info` dictionary. In addition, the action, and its noisy version are also available, with shape `(batch_size, n_muscles)`. Additionally, the goal attribute, which is held at the `Environment` level, is also returned.

The observation vector dimensionality is `(batch_size, n_observations)`, with the second dimension being an arbitrary combination of `Effector` states and other information that the user deems relevant for the network to receive as input. These observations can be potentially noised and/or time-delayed. By default, this is the goal, the fingertip state, the normalized muscle length for each muscle and normalized muscle elocity for each muscle. Since there are five muscles in this effector, and the goal and fingertip are 2D cartesian position vectors, this yields `n_observations = 2 + 2 + 5 + 5 = 14`. The content of the observation vector, and whether noise and/or time-delay is applied is defined when building the `Environment` class. This is further detailed in the follow-up `3-environments.ipynb` tutorial.

**Importantly**, users familiar with reinforcement learning (RL) toolboxes will notice that this API design matches the standardized API used for RL packages and pipelines. This is not coincidental. The RL community has been relying on open-source packages and cross-compatible pipelines for years now, and abiding by a similar API design ensures that the insights and best practices learnt through this experience is translated here. Another important feature is that it enables compatibility between this package and packages implementing state-of-the-art RL agents.
